In [1]:
from cameo import load_model
from os import getcwd
from Functions_Modules import curation_tools as ct
from math import isnan
from numpy import zeros

/Users/clie/.virtualenvs/modeling/lib/python2.7/site-packages/cobra/io/sbml3.py:24: UserWarning: Install lxml for faster SBML I/O
  warn("Install lxml for faster SBML I/O")


In [2]:
import pandas as pd
import numpy as np

In [3]:
relative_directory = getcwd()
model = load_model(relative_directory + '/Reconstructions/MethylococcusModel10.json')

# Base Settings of the Model

What is the current state?

In [4]:
model.medium

,reaction_id,reaction_name,lower_bound,upper_bound
0,EX_co2_c,CO(2) exchange,-4.935116,1000.000000
1,EX_pi_c,phosphate exchange,-1000.000000,1000.000000
2,EX_o2_c,dioxygen exchange,-194.443565,1000.000000
3,EX_nh3_c,NH4(+) exchange,-1000.000000,1000.000000
4,EX_so4_c,sulfate exchange,-1000.000000,1000.000000
5,EX_ch4_p,methane exchange,0.100000,108.572549
6,EX_cobalt2_c,Co2+ exchange,-1000.000000,1000.000000


In [5]:
print model.objective

Maximize
-1.0*BIOMASS_REACTION_reverse_42726 + 1.0*BIOMASS_REACTION


In [6]:
model.reactions.pMMO_dir_coupl_im

Id,pMMO_dir_coupl_im
Name,particulate Methane Monooxygenase
Stoichiometry,ch4_p + 2 focytcc555_p + o2_p --> 2 ficytcc555_p + h2o_p + meoh_p
GPR,(((MCA1796 and MCA1797 and MCA1798) or (MCA2853 and MCA2854 and MCA2855)) and MCA0295)
Lower bound,0.000000
Upper bound,1000.000000


In [7]:
model.reactions.sMMO_c

Id,sMMO_c
Name,soluble Methane Monooxygenase
Stoichiometry,ch4_c + h_c + nadh_c + o2_c --> h2o_c + meoh_c + nad_c
GPR,( MCA1194 and MCA1195 and MCA1198 and MCA1196 and MCA1200 and MCA1202 and MCA1205 )
Lower bound,0.000000
Upper bound,1000.000000


In [8]:
model.reactions.pMMO_im

Id,pMMO_im
Name,particulate Methane Monooxygenase
Stoichiometry,ch4_p + o2_p + q8h2_im --> h2o_p + meoh_p + q8_im
GPR,(((MCA1796 and MCA1797 and MCA1798) or (MCA2853 and MCA2854 and MCA2855)) and MCA0295)
Lower bound,0.000000
Upper bound,0.000000


In [9]:
model.reactions.BIOMASS_REACTION.change_bounds(0.1,1000)

In [10]:
model.solve().f

2.6066282676295067

Change the state to be neither Carbon nor Oxygen limited, and fix up the direction of the CH4 Exchange.

In [11]:
model.reactions.EX_o2_c.change_bounds(-1000,1000)

In [12]:
ct.invert_reaction(model.reactions.EX_ch4_p)

In [13]:
model.reactions.EX_ch4_p.change_bounds(-1000,-0.1)

In [14]:
model.reactions.EX_co2_c.change_bounds(0,1000)

Turn off the other modes of Electron transport too and operate on the default i.e. commonly accepted state in a pMMO-only environment.

In [15]:
model.reactions.pMMO_dir_coupl_im.upper_bound = 0

In [16]:
model.reactions.sMMO_c.upper_bound = 0

In [17]:
model.reactions.CYOR_q8_im.change_bounds(0, 1000)

In [18]:
model.reactions.pMMO_im.upper_bound = 1000

In [19]:
model.solve().f

6.002042639159091

# Fixes of reaction directionalities

I noticed that NQR didn't pump Na out of the cytosol like it should.

In [20]:
model.reactions.NQR_im.add_metabolites(
{model.metabolites.na1_c: -2.0,
 model.metabolites.na1_p: 2.0,
 model.metabolites.nad_c: 1.0,
 model.metabolites.nadh_c: -1.0,
 model.metabolites.q8h2_im: 1.0,
 model.metabolites.q8_im: -1.0,
 model.metabolites.h_c: -1.0}, combine=False)

In [21]:
model.reactions.NQR_im

Id,NQR_im
Name,Na(+)-translocating NADH-quinone - NQR
Stoichiometry,h_c + 2.0 na1_c + nadh_c + q8_im --> 2.0 na1_p + nad_c + q8h2_im
GPR,(MCA2389 and MCA2388 and MCA2387 and MCA2386 and MCA2385 and MCA2384)
Lower bound,0.000000
Upper bound,1000.000000


Enolase is supposed to be reversible. I made that change so that 3pg can both be produced from Pyruvate and G3p.

In [22]:
model.reactions.ENO_c.lower_bound = -1000

The publication 'Characterization of the pyrophosphate-dependent 6-phosphofructokinase from Methylococcuscapsulatus Bath' by Reshetnikov et al. thoroughly characterizes MCA1251 as an exclusively ppi-dependent kinase. Km and Vmax have been determined in the paper. Hence, modifications to the reactions in the model should be made:

In [23]:
model.remove_reactions([model.reactions.PFK_adp_c,model.reactions.PFK_c,model.reactions.PFK_2_c,model.reactions.FBP_c])

In [24]:
model.reactions.PFK_ppi_c.notes = {'CONFIDENCE SCORE': '4','COFACTOR': ['Mg2+','Co2+','Mn2+'],'LOCALIZATION':'Cytosol'}

In [25]:
# model.reactions.FBP_c.notes = {'CONFIDENCE SCORE': '4','COFACTOR': ['Mg2+','Co2+','Mn2+'],'LOCALIZATION':'Cytosol'}

In [26]:
from cameo import Reaction,Metabolite

In [27]:
rxn = Reaction(id='PFK_3_ppi_c',name='Pyrophosphate-dependent Phosphofructokinase (s7p)',
              lower_bound = -1000, upper_bound = 1000)
rxn.add_metabolites({model.metabolites.s7p_c:-1,
                    model.metabolites.ppi_c:-1,
                    model.metabolites.pi_c:1,
                    model.metabolites.s17bp_c:1,
                    model.metabolites.h_c:1})
# Check if RXN is mass and charge balanced!
print (rxn.check_mass_balance())
rxn.notes = {'CONFIDENCE SCORE': '4','COFACTOR': ['Mg2+','Co2+','Mn2+'],'LOCALIZATION':'Cytosol'}
model.add_reaction(rxn)

{}


Another paper 'Characterization of Recombinant Fructose 1 , 6 Bisphosphate Aldolase from Methylococcus capsulatus Bath' by Rozova et al. indicates that FBA can catalyze both the reversible cleavage of fdp aswell as s17pb and thus play an important role in the carbon rearrangements of the Calvin Cycle in M. capsulatus.

In [28]:
model.reactions.FBA3_c.lower_bound = -1000

In [29]:
model.solve().f

6.002042639159084

## ATP

Let's quickly double-check to see if there are reactions left that produce ATP that shouldnt!

In [30]:
model.solve().f

6.002042639159084

In [31]:
[rxn for rxn in model.metabolites.atp_c.reactions if (model.metabolites.atp_c in rxn.products or rxn.reversibility) and abs(rxn.flux) > 0]

[<Reaction NDPK4_c at 0x11c702190>,
 <Reaction FMNAT_c at 0x11d1de510>,
 <Reaction HSK_GAPFILLING_c at 0x11d27a810>,
 <Reaction NDPK1_c at 0x11cf42910>,
 <Reaction DADK_c at 0x11cf0aad0>,
 <Reaction GLUTRS_c at 0x11cd5f2d0>,
 <Reaction AICART2_c at 0x11ccdf290>,
 <Reaction CYTK1_c at 0x11d081890>,
 <Reaction ACCOAC_c at 0x11cc96d10>,
 <Reaction GLYCK_c at 0x11d04b510>,
 <Reaction HEX1_c at 0x11cc8b910>,
 <Reaction PNTK_c at 0x11d1e3b10>,
 <Reaction NADK_c at 0x11d1e7c10>,
 <Reaction URIDK2m_c at 0x11c701cd0>,
 <Reaction FACOAL160_c at 0x11cfd7a50>,
 <Reaction ATPSh_im at 0x11d26de50>,
 <Reaction DGK1_c at 0x11cf42510>]

Fixed based on BiGG Database specifically iJO1366 model wherever possible

In [32]:
ct.invert_reaction(model.reactions.DGK1_c)

In [33]:
model.reactions.NADK_c.change_bounds(0,1000)

In [34]:
model.reactions.ACCOAC_c.change_bounds(0,1000)

In [35]:
ct.invert_reaction(model.reactions.FACOAL160_c)

In [36]:
model.reactions.GLUTRS_c.change_bounds(0,1000)

In [37]:
ct.invert_reaction(model.reactions.DADK_c)

In [38]:
ct.invert_reaction(model.reactions.SUCOAS_c)

In [39]:
model.reactions.FMNAT_c.change_bounds(0,1000)

In [40]:
ct.invert_reaction(model.reactions.CYTK1_c)

In [41]:
model.reactions.AICART2_c.change_bounds(0,1000)

In [42]:
model.reactions.PNTK_c.change_bounds(0,1000)

In [43]:
model.reactions.HSK_GAPFILLING_c.change_bounds(0,1000)

In [44]:
ct.invert_reaction(model.reactions.URIDK2m_c)

Reaction GLYCK_c can't be set to be reversible because there is no other reaction producing glyc_R which is required in the biomass reaction adapted from de la Torre et al.

In [45]:
model.solve().f

6.002042639159139

More thorough check, when we change the model objective to produce ATP instead of BIOMASS.

In [46]:
model.remove_reactions([model.reactions.NGAM_c])

In [47]:
model.change_objective(model.reactions.EX_atp_c)

In [48]:
model.reactions.ATPM_c.change_bounds(1000,1000)

In [49]:
model.solve().f

32.77796684666714

In [50]:
for rxn in model.metabolites.atp_c.reactions:
    if rxn.flux > 200:
        print rxn.id,rxn.flux

ADCPS1_c 897.28880266
ATPM_c 1000.0
ATPSh_im 500.0


In [51]:
ct.invert_reaction(model.reactions.ADNK1_c)

In [52]:
model.solve().f

32.77796684666714

In [53]:
for rxn in model.metabolites.atp_c.reactions:
    if rxn.flux > 200:
        print rxn.id,rxn.flux

ADCPS1_c 897.28880266
ATPM_c 1000.0
ATPSh_im 500.0


In [54]:
ct.invert_reaction(model.reactions.ADCPS1_c)

In [55]:
model.solve().f

32.77796684666681

In [56]:
for rxn in model.metabolites.atp_c.reactions:
    if rxn.flux > 200:
        print rxn.id,rxn.flux

ADK1_c 685.294425807
PYK_c 620.455591013
PPDK_c 619.584228713
ATPM_c 1000.0
CYTK2_c 1000.0
ATPSh_im 1000.0


In [57]:
ct.invert_reaction(model.reactions.CYTK2_c)

In [58]:
model.solve().f

32.77796684666658

In [59]:
for rxn in model.metabolites.atp_c.reactions:
    if rxn.flux > 200:
        print rxn.id,rxn.flux

DGNSKm_c 488.502723613
ATPM_c 1000.0
ATPSh_im 875.0


In [60]:
ct.invert_reaction(model.reactions.DGNSKm_c)

In [61]:
model.reactions.DGNSKm_c.lower_bound = 0

In [62]:
model.solve().f

21.204966999729393

In [63]:
for rxn in model.metabolites.atp_c.reactions:
    if rxn.flux > 200:
        print rxn.id,rxn.flux

ATPSNa_im 236.078884297
ATPM_c 1000.0
ATPSh_im 1000.0


Now we have removed reactions that can run in a cycle to replenish ATP bypassing the ATPase. The proof for this
is that when we optimize the production of ATP and at the same time have an insanely high turnover of ATP when the model is carbon limited, the solution will be infeasible.

In [64]:
model.reactions.EX_ch4_p.change_bounds(-20,-0.1)

I am setting the NGAM requirement to a value calculated for EColi, This is also used by De la torre et all and brought up in the Palson Protocol.

In [65]:
model.reactions.ATPM_c.change_bounds(8.390000,8.390000)

In [66]:
model.reactions.ATPM_c.name = 'ATP maintenance requirement'

In [67]:
model.reactions.ATPM_c.gene_reaction_rule = ''

In [68]:
model.solve().f

0.7779668466666615

In [69]:
model.reactions.EX_ch4_p.change_bounds(-1000,-0.1)

## NH3

Let's double-check if NH3 can only be taken up through the known main two systems: GS/GOGAT (GLNS + GLUSx/GLUSy) or Alanine Dehydrogenase (ALAD_\_L/ALAD_\_Ly + ALATA_L)

In [70]:
[rxn for rxn in model.metabolites.nh3_c.reactions if rxn.reversibility or model.metabolites.nh3_c in rxn.reactants]

[<Reaction GLNS_c at 0x11d04b050>,
 <Reaction ALAD__L_c at 0x11d009c10>,
 <Reaction NH3_im at 0x11d2b7990>,
 <Reaction GMPS_c at 0x11cf68a50>,
 <Reaction AMOs_c at 0x11d2b7310>,
 <Reaction GCCb_c at 0x11d02cad0>,
 <Reaction ASNS2_c at 0x11cd9b690>,
 <Reaction CTPS1_c at 0x11d212c10>,
 <Reaction NADS1_c at 0x11ccdf690>,
 <Reaction EX_nh3_c at 0x11cd46cd0>,
 <Reaction ALAD__Ly_c at 0x11d2be550>,
 <Reaction DORNOp_c at 0x11cd97690>]

In [71]:
model.reactions.ASNS2_c

Id,ASNS2_c
Name,1 H(+) + 1 diphosphate(3-) + 1 AMP + 1 L-asparagine zwitterion = 1 NH4(+) + 1 ATP + 1 L-aspartate(1-)
Stoichiometry,asp__L_c + atp_c + nh3_c --> amp_c + asn__L_c + h_c + ppi_c
GPR,MCA0472 or MCA0542 or MCA2127
Lower bound,0.000000
Upper bound,1000.000000


Glycine Cleavage Complex was reversed according to the reactions in iHN637 and in agreement with the above
information from literature.

In [72]:
ct.invert_reaction(model.reactions.GCCa_c)

In [73]:
ct.invert_reaction(model.reactions.GCCb_c)

In [74]:
ct.invert_reaction(model.reactions.GCCc_c)

ASNS2_c was removed because it bypassed ASNS1_c which catalyzed the same reaction but using Glutamine, which agrees with literature.

In [75]:
model.remove_reactions([model.reactions.ASNS2_c])

NADS1_c was removed because it bypassed NADS2_c which catalyzed the same reaction but using Glutamine, which agrees with literature.

In [76]:
model.remove_reactions([model.reactions.NADS1_c])

DORNOp seems to be a eukaryotic reaction (as it is only represented on BiGG in the human, cho or mouse models. In all three of them it is not reversible and actually NH3 is a product along with H2O2 and 5a2opntn.

In [77]:
ct.invert_reaction(model.reactions.DORNOp_c)

MCA0290 is annotated as GMP synthase [glutamine-hydrolyzing]. Hence GMPS_c will be removed while keeping GMPS2.

In [78]:
model.remove_reactions([model.reactions.GMPS_c])

MCA2513 is annotated as CTP synthase. On Uniprot its reaction is specified as 
ATP + UTP + L-glutamine = ADP + phosphate + CTP + L-glutamate

CTPS1 will be removed, CTPS2 will be kept.

In [79]:
model.remove_reactions([model.reactions.CTPS1_c])

In [80]:
model.solve().f

32.7779668466663

In [81]:
model.change_objective(model.reactions.BIOMASS_REACTION)

## NADP/NAD

In [82]:
[rxn for rxn in model.metabolites.nadh_c.reactions if rxn.reversibility or model.metabolites.nadh_c in rxn.products]

[<Reaction UDPGD_c at 0x11d07d050>,
 <Reaction G3PD1_c at 0x11cc94090>,
 <Reaction ALDD1_c at 0x11d26d150>,
 <Reaction DHORD_NAD_c at 0x11ccdf450>,
 <Reaction GAPD_c at 0x11d1e5190>,
 <Reaction GLUDx_c at 0x11d0159d0>,
 <Reaction GLYCLTDx_c at 0x11cce2cd0>,
 <Reaction PPND_c at 0x11cf7e310>,
 <Reaction IMPD_c at 0x11cf689d0>,
 <Reaction HISTD_GAPFILLING_c at 0x11d27ab10>,
 <Reaction EAR100x_c at 0x11d00eb50>,
 <Reaction RDH1_c at 0x11ccdfc90>,
 <Reaction ME1_c at 0x11cc96b90>,
 <Reaction DHFR2i_c at 0x11cc9bbd0>,
 <Reaction PGCD_GAPFILLING_c at 0x11d27a750>,
 <Reaction AKGDH_c at 0x11cce6610>,
 <Reaction GCCc_c at 0x11d02ccd0>,
 <Reaction 34DHOXPEGOX_c at 0x11cf89550>,
 <Reaction IPMD_c at 0x11d1d7b90>,
 <Reaction GLYCL_c at 0x11cc99cd0>,
 <Reaction MDH_c at 0x11d0b4a10>,
 <Reaction FDH_c at 0x11cc995d0>,
 <Reaction ICDHxm_c at 0x11d1c2b90>,
 <Reaction NADTRHD_c at 0x11d1e7e10>,
 <Reaction ALCD2if_c at 0x11d1fc690>,
 <Reaction PDH_c at 0x11cce23d0>,
 <Reaction VALDHr_c at 0x11ccdd110>,

In [83]:
model.reactions.EAR100y_c.lower_bound = 0
model.reactions.EAR100x_c.lower_bound = 0

Inverted SHCHF based on iJO1366 directionality of the reaction

In [84]:
ct.invert_reaction(model.reactions.SHCHF_c)

In [85]:
ct.invert_reaction(model.reactions.GAPDH_nadp_hi_c)

In [86]:
model.reactions.GAPDH_nadp_hi_c.lower_bound = 0

In [87]:
model.reactions.NADTRHD_c.lower_bound = 0

In [88]:
model.reactions.GLYCLTDx_c.lower_bound = 0

In [89]:
model.reactions.GLYCLTDy_c.lower_bound = 0

In [90]:
ct.invert_reaction(model.reactions.get_by_id('34DHOXPEGOX_c'))

In [91]:
model.reactions.DHFR2i_c.lower_bound = 0

In [92]:
model.reactions.HSDx_c.lower_bound = 0

In [93]:
model.remove_reactions([model.reactions.RDH1_c,model.reactions.RDH1a_c],remove_orphans=True)

In [94]:
model.remove_reactions([model.reactions.EX_retinol_c,model.reactions.EX_retinal_c],remove_orphans=True)

In [95]:
model.reactions.DHFOR_c.lower_bound = 0

In [96]:
model.reactions.FOLR2_c.lower_bound = 0

In [97]:
[rxn for rxn in model.metabolites.nadph_c.reactions if rxn.reversibility]

[<Reaction ALCD2y_c at 0x11cce3d50>,
 <Reaction DDSMSTOLR_c at 0x11cfd7310>,
 <Reaction 3OACOAR_c at 0x11cd970d0>,
 <Reaction PPND2_c at 0x11cd9b510>,
 <Reaction GLUTRR_c at 0x11cd5f1d0>,
 <Reaction 3OAR140_c at 0x11d0226d0>,
 <Reaction 3OAR120_c at 0x11d00ed50>,
 <Reaction 3OAR100_c at 0x11d00e8d0>,
 <Reaction SQLS_c at 0x11cd97a50>,
 <Reaction ZYMSTR_c at 0x11cfc3c50>,
 <Reaction 3OAR80_c at 0x11cffbd90>,
 <Reaction MDHy_c at 0x11cce8dd0>,
 <Reaction GLUSy_c at 0x11d015250>,
 <Reaction 3OAR60_c at 0x11cffb510>,
 <Reaction 3OAR160_c at 0x11d022e90>,
 <Reaction 3OAR40_1_c at 0x11cfe9a10>,
 <Reaction MTHFD_GAPFILLING_c at 0x11d27ac50>]

In [98]:
model.reactions.DDSMSTOLR_c.lower_bound = 0

In [99]:
model.reactions.SQLS_c.lower_bound = 0

In [100]:
model.reactions.ZYMSTR_c.lower_bound = 0

In [101]:
model.reactions.GLUTRR_c.lower_bound = 0

In [102]:
model.reactions.GLUSy_c.lower_bound = -1000

In [103]:
ct.invert_reaction(model.reactions.ALCD2y_c)

In [104]:
ct.invert_reaction(model.reactions.ALCD2if_c)

In [105]:
model.reactions.ALCD2y_c.lower_bound = 0

In [106]:
model.reactions.ALCD2if_c.lower_bound = 0

In [107]:
ct.invert_reaction(model.reactions.PPND2_c)

In [108]:
model.reactions.PPND2_c.lower_bound = 0

In [109]:
model.metabolites.ahdt_c.reactions

frozenset({<Reaction AKP1_c at 0x11d1dc5d0>,
           <Reaction PTHPS_c at 0x11d1dc750>,
           <Reaction 6CTDS_c at 0x11d1dca90>,
           <Reaction EX_ahdt_c at 0x11d232590>})

## Tetrahydrofolate Biosynthesis Fix

When investigating the above reactions I noticed that the model can't produce Tetrahydrofolate, because the inital step of the pathway is missing. MCA1670 is annotated as a GTP cyclohydrolase hence the reaction will be added.

In [110]:
rxn = Reaction()
rxn.name = 'GTP cyclohydrolase I'
rxn.subsystem = 'Folate biosynthesis'
rxn.lower_bound = 0.
rxn.upper_bound = 1000.
rxn.add_metabolites({model.metabolites.gtp_c:-1,
                    model.metabolites.h2o_c:-1,
                    model.metabolites.ahdt_c:1,
                    model.metabolites.for_c:1,
                    model.metabolites.h_c:1})
# Check if RXN is mass and charge balanced!
print (rxn.check_mass_balance())
# Update ID to BiGG
rxn.id = 'GTPCI'
model.add_reaction(rxn)

{}


In [111]:
model.reactions.GTPCI.gene_reaction_rule = 'MCA1670'

In [112]:
ct.invert_reaction(model.reactions.ADCL_c)

In [113]:
ct.invert_reaction(model.reactions.DHFS_c)

In [114]:
model.change_objective(model.reactions.BIOMASS_REACTION)

In [115]:
model.solve().f

5.981080383363763

In [116]:
model.change_objective(model.reactions.BIOMASS_REACTION)

# Are there any genes without a reaction?

In [117]:
genes_without_reactions = []
for gene in model.genes:
    if not gene.reactions:
        genes_without_reactions.append(gene.id)

In [118]:
len(genes_without_reactions)

61

Yup. Quite a lot. In order to quickly identify relevant ones, I downloaded a recent mapping from Uniprot (Date 20170216). Locus Tag vs EC Number.

In [119]:
path = '/Users/clie/Desktop/EFPro2/Genome/UniProtExport/MCA_GeneLocus_vs_EC_Number_20170216.tab'

In [120]:
locus_tag_EC_map = pd.read_csv(path,sep='\t')

In [121]:
locus_tag_EC_map = locus_tag_EC_map[[u'Gene names  (ordered locus )',u'EC number']]

In [122]:
df = locus_tag_EC_map.dropna(subset=[u'Gene names  (ordered locus )']) 

In [123]:
df1 = pd.concat([pd.Series(row[u'EC number'], row[u'Gene names  (ordered locus )'].split(';'))              
                    for _, row in df.iterrows()]).reset_index()

In [124]:
df2 = pd.concat([pd.Series(row[0], row['index'].split(' '))              
                    for _, row in df1.iterrows()]).reset_index()

In [125]:
df2['index'].replace('', np.nan, inplace=True)

In [126]:
df3 = df2.dropna(subset=['index']) 

In [127]:
df4 = df3.dropna()

In [128]:
EC_dict = df4.set_index('index').to_dict(orient='dict')

In [129]:
met_genes = {x:EC_dict[0][x] for x in genes_without_reactions if x in EC_dict[0].keys()}

In [130]:
non_met_genes = [x for x in genes_without_reactions if x not in EC_dict[0].keys()]

In [131]:
print len(met_genes),len(non_met_genes)

45 16


There are about 45 genes for which an EC Number can be found in Uniprot and 16 which lack this annotation.

In [132]:
met_genes

{u'MCA0048': '2.7.1.107',
 u'MCA0067': '2.4.1.1',
 u'MCA0074': '2.5.1.18',
 u'MCA0109': '2.3.1.181',
 u'MCA0110': '2.8.1.8',
 u'MCA0165': '1.12.7.2',
 u'MCA0166': '1.12.7.2',
 u'MCA0229': '1.18.6.1',
 u'MCA0230': '1.18.6.1',
 u'MCA0231': '1.18.6.1',
 u'MCA0296': '3.2.1.-',
 u'MCA0603': '1.1.1.262',
 u'MCA0720': '2.7.7.7',
 u'MCA0799': '4.2.1.109',
 u'MCA1066': '2.7.7.6',
 u'MCA1067': '2.7.7.6',
 u'MCA1077': '2.8.1.10',
 u'MCA1125': '2.8.1.6',
 u'MCA1128': '2.1.1.197',
 u'MCA1232': '4.3.99.3',
 u'MCA1233': '6.3.4.20',
 u'MCA1286': '2.7.7.7',
 u'MCA1303': '2.5.1.18',
 u'MCA1309': '2.7.7.8',
 u'MCA1328': '2.7.7.7',
 u'MCA1451': '2.7.7.7',
 u'MCA1473': '2.4.1.25',
 u'MCA1475': '2.4.1.18',
 u'MCA1476': '2.4.1.21',
 u'MCA1555': '2.8.1.2',
 u'MCA1667': '2.6.99.2',
 u'MCA1691': '2.5.1.75',
 u'MCA1737': '3.4.11.5',
 u'MCA1740': '6.2.1.5',
 u'MCA1741': '6.2.1.5',
 u'MCA1995': '2.7.7.7',
 u'MCA2022': '2.7.7.6',
 u'MCA2220': '3.5.2.6',
 u'MCA2347': '2.7.7.6',
 u'MCA2540': '2.4.1.1',
 u'MCA2573': '

In [133]:
inv_met_genes = {}
for k, v in met_genes.iteritems():
    if v not in inv_met_genes.keys():
        inv_met_genes[v] = k
    else:
        inv_met_genes[v] = inv_met_genes[v] + ' or ' + k  

In [134]:
inv_met_genes

{'1.1.1.262': u'MCA0603',
 '1.12.7.2': u'MCA0165 or MCA0166',
 '1.14.13.70': u'MCA2711',
 '1.18.6.1': u'MCA0229 or MCA0230 or MCA0231',
 '2.1.1.197': u'MCA1128',
 '2.3.1.181': u'MCA0109',
 '2.4.1.1': u'MCA0067 or MCA2540',
 '2.4.1.18': u'MCA1475',
 '2.4.1.21': u'MCA1476 or MCA2606',
 '2.4.1.25': u'MCA1473',
 '2.5.1.18': u'MCA0074 or MCA1303',
 '2.5.1.75': u'MCA1691',
 '2.6.99.2': u'MCA1667',
 '2.7.1.107': u'MCA0048',
 '2.7.7.6': u'MCA1067 or MCA1066 or MCA2022 or MCA2347',
 '2.7.7.7': u'MCA1286 or MCA1328 or MCA1995 or MCA3032 or MCA2573 or MCA1451 or MCA0720',
 '2.7.7.8': u'MCA1309',
 '2.8.1.10': u'MCA1077',
 '2.8.1.2': u'MCA1555',
 '2.8.1.6': u'MCA1125',
 '2.8.1.8': u'MCA0110',
 '3.2.1.-': u'MCA0296',
 '3.4.11.5': u'MCA1737',
 '3.5.1.11': u'MCA3025',
 '3.5.2.6': u'MCA2220',
 '4.2.1.109': u'MCA0799',
 '4.3.99.3': u'MCA1232',
 '6.2.1.5': u'MCA1740 or MCA1741',
 '6.3.4.20': u'MCA1233'}

Perhaps there are genes with EC numbers that match those already in the model:

In [135]:
EC_numbers_RXN_dict = {}

for rxn in model.reactions:
    if 'brenda' in rxn.annotation:
        EC_numbers_RXN_dict[rxn.id] = rxn.annotation[u'brenda']

In [136]:
for key in met_genes:
    for rxn in EC_numbers_RXN_dict:
        if unicode(met_genes[key], "utf-8") in EC_numbers_RXN_dict[rxn]:
            print key,met_genes[key],rxn,EC_numbers_RXN_dict[rxn]

MCA1740 6.2.1.5 ITCOALm_c 6.2.1.5
MCA1740 6.2.1.5 SUCOAS_c 6.2.1.5
MCA1741 6.2.1.5 ITCOALm_c 6.2.1.5
MCA1741 6.2.1.5 SUCOAS_c 6.2.1.5
MCA1475 2.4.1.18 LPADSS_c 2.4.1.182
MCA1067 2.7.7.6 ACBIPGT_c 2.7.7.62
MCA1066 2.7.7.6 ACBIPGT_c 2.7.7.62
MCA2022 2.7.7.6 ACBIPGT_c 2.7.7.62
MCA2347 2.7.7.6 ACBIPGT_c 2.7.7.62
MCA0067 2.4.1.1 LPADSS_c 2.4.1.182
MCA2540 2.4.1.1 LPADSS_c 2.4.1.182


Only two genes. Well.

In [137]:
model.reactions.SUCOAS_c.gene_reaction_rule = '(MCA0967 and MCA0968) or (MCA1740 and MCA1741)'
model.reactions.ITCOALm_c.gene_reaction_rule = '(MCA0967 and MCA0968) or (MCA1740 and MCA1741)'

In [138]:
model.remove_reactions([model.reactions.SUCOAS1m_c])
model.solve().f

5.981080383363871

In [139]:
inv_met_genes.pop('6.2.1.5')

u'MCA1740 or MCA1741'

In [140]:
len(inv_met_genes)

28

Exploration of the remaining EC numbers:

In [141]:
inv_met_genes

{'1.1.1.262': u'MCA0603',
 '1.12.7.2': u'MCA0165 or MCA0166',
 '1.14.13.70': u'MCA2711',
 '1.18.6.1': u'MCA0229 or MCA0230 or MCA0231',
 '2.1.1.197': u'MCA1128',
 '2.3.1.181': u'MCA0109',
 '2.4.1.1': u'MCA0067 or MCA2540',
 '2.4.1.18': u'MCA1475',
 '2.4.1.21': u'MCA1476 or MCA2606',
 '2.4.1.25': u'MCA1473',
 '2.5.1.18': u'MCA0074 or MCA1303',
 '2.5.1.75': u'MCA1691',
 '2.6.99.2': u'MCA1667',
 '2.7.1.107': u'MCA0048',
 '2.7.7.6': u'MCA1067 or MCA1066 or MCA2022 or MCA2347',
 '2.7.7.7': u'MCA1286 or MCA1328 or MCA1995 or MCA3032 or MCA2573 or MCA1451 or MCA0720',
 '2.7.7.8': u'MCA1309',
 '2.8.1.10': u'MCA1077',
 '2.8.1.2': u'MCA1555',
 '2.8.1.6': u'MCA1125',
 '2.8.1.8': u'MCA0110',
 '3.2.1.-': u'MCA0296',
 '3.4.11.5': u'MCA1737',
 '3.5.1.11': u'MCA3025',
 '3.5.2.6': u'MCA2220',
 '4.2.1.109': u'MCA0799',
 '4.3.99.3': u'MCA1232',
 '6.3.4.20': u'MCA1233'}

'1.1.1.262': 'MCA0603' = Catalyzes the NAD(P)-dependent oxidation of 4-(phosphohydroxy)-L-threonine (HTP) into 2-amino-3-oxo-4-(phosphohydroxy)butyric acid which spontaneously decarboxylates to form 3-amino-2-oxopropyl phosphate (AHAP)

4-phosphonooxy-L-threonine + NAD+ = 3-amino-2-oxopropyl phosphate + CO2 + NADH.

http://www.uniprot.org/uniprot/Q3V8A3

'1.12.7.2': u'MCA0165 or MCA0166' = Nickel-iron hydrogenase

http://www.uniprot.org/uniprot/Q60CE2

'1.14.13.70': u'MCA2711'= Cytochrome P450 51

Also known as: 
1. Lanosterol 14-alpha-demethylase.
2. Lanosterol 14-demethylase.
3. Obtusufoliol 14-demethylase.

A 14-alpha-methylsteroid + 3 O(2) + 3 NADPH <=> a Delta(14)-steroid + formate + 3 NADP(+) + 4 H(2)O

http://www.uniprot.org/uniprot/Q603T8

'1.18.6.1': u'MCA0229 or MCA0230 or MCA0231' = Nitrogenase
    
8 reduced ferredoxin + 8 H+ + N2 + 16 ATP + 16 H2O = 8 oxidized ferredoxin + H2 + 2 NH3 + 16 ADP + 16 phosphate

http://www.uniprot.org/uniprot/Q60C81

'2.1.1.197': 'MCA1128' = Malonyl-[acyl-carrier protein] O-methyltransferase

This protein is involved in the pathway biotin biosynthesis, which is part of Cofactor biosynthesis.
    
S-adenosyl-L-methionine + malonyl-[acyl-carrier protein] = S-adenosyl-L-homocysteine + malonyl-[acyl-carrier protein] methyl ester

http://www.uniprot.org/uniprot/Q609U9

'2.3.1.181': 'MCA0109' = Octanoyltransferase

Catalyzes the transfer of endogenously produced octanoic acid from octanoyl-acyl-carrier-protein onto the lipoyl domains of lipoate-dependent enzymes. Lipoyl-ACP can also act as a substrate although octanoyl-ACP is likely to be the physiological substrate

This subpathway is part of the pathway protein lipoylation via endogenous pathway, which is itself part of Protein modification.

Octanoyl-[acyl-carrier-protein] + protein = protein N(6)-(octanoyl)lysine + [acyl-carrier-protein].

http://www.uniprot.org/uniprot/Q60CJ7

'2.4.1.1': 'MCA0067 or MCA2540' = Alpha-1,4 glucan phosphorylase

Phosphorylase is an important allosteric enzyme in carbohydrate metabolism. Enzymes from different sources differ in their regulatory mechanisms and in their natural substrates. However, all known phosphorylases share catalytic and structural properties

((1->4)-alpha-D-glucosyl)(n) + phosphate = ((1->4)-alpha-D-glucosyl)(n-1) + alpha-D-glucose 1-phosphate

http://www.uniprot.org/uniprot/Q604J9

'2.4.1.18': 'MCA1475' = 1,4-alpha-glucan branching enzyme GlgB

Catalyzes the formation of the alpha-1,6-glucosidic linkages in glycogen by scission of a 1,4-alpha-linked oligosaccharide from growing alpha-1,4-glucan chains and the subsequent attachment of the oligosaccharide to the alpha-1,6 position.

Transfers a segment of a (1->4)-alpha-D-glucan chain to a primary hydroxy group in a similar glucan chain

http://www.uniprot.org/uniprot/Q608L5

'2.4.1.21': u'MCA1476 or MCA2606' =  Glycogen synthase 1

ADP-glucose + (1,4-alpha-D-glucosyl)(n) = ADP + (1,4-alpha-D-glucosyl)(n+1).

This protein is involved in the pathway glycogen biosynthesis, which is part of Glycan biosynthesis.

http://www.uniprot.org/uniprot/Q608L4

'2.4.1.25': 'MCA1473' = 4-alpha-glucanotransferase

ransfers a segment of a (1->4)-alpha-D-glucan to a new position in an acceptor, which may be glucose or a (1->4)-alpha-D-glucan

http://www.uniprot.org/uniprot/Q608L7

'2.5.1.18': u'MCA0074 or MCA1303' = Glutathione S-transferase

RX + glutathione <=> HX + R-S-glutathione

http://www.uniprot.org/uniprot/Q60CN1

'2.5.1.75': u'MCA1691' = tRNA dimethylallyltransferase 
Catalyzes the transfer of a dimethylallyl group onto the adenine at position 37 in tRNAs that read codons beginning with uridine, leading to the formation of N6-(dimethylallyl)adenosine (i6A).

Dimethylallyl diphosphate + adenine(37) in tRNA = diphosphate + N(6)-dimethylallyladenine(37) in tRNA

http://www.uniprot.org/uniprot/Q607R4

'2.6.99.2': 'MCA1667' = Pyridoxine 5'-phosphate synthase

Catalyzes the complicated ring closure reaction between the two acyclic compounds 1-deoxy-D-xylulose-5-phosphate (DXP) and 3-amino-2-oxopropyl phosphate (1-amino-acetone-3-phosphate or AAP) to form pyridoxine 5'-phosphate (PNP) and inorganic phosphate

1-deoxy-D-xylulose 5-phosphate + 3-amino-2-oxopropyl phosphate = pyridoxine 5'-phosphate + phosphate + 2 H2O

http://www.uniprot.org/uniprot/Q3V8A2

'2.7.1.107': 'MCA0048' = Diacylglycerol kinase, dgkA

Involved in synthesis of membrane phospholipids and the neutral lipid triacylglycerol.
Activity is stimulated by certain phospholipids.
In plants and animals the product 1,2-diacyl-sn-glycerol 3-phosphate is an important second messenger.

ATP + 1,2-diacyl-sn-glycerol <=> ADP + 1,2-diacyl-sn-glycerol 3-phosphate

http://www.uniprot.org/uniprot/Q60CV3

'2.7.7.6': 'MCA1067 or MCA1066 or MCA2022 or MCA2347' = DNA-directed RNA polymerase subunit beta'

DNA-dependent RNA polymerase catalyzes the transcription of DNA into RNA using the four ribonucleoside triphosphates as substrates.

Nucleoside triphosphate + RNA(n) = diphosphate + RNA(n+1).

http://www.uniprot.org/uniprot/Q60A05

'2.7.7.7': u'MCA1286 or MCA1328 or MCA1995 or MCA3032 or MCA2573 or MCA1451 or MCA0720' = DNA-directed DNA polymerase

Deoxynucleoside triphosphate + DNA(n) = diphosphate + DNA(n+1).

http://www.uniprot.org/uniprot/Q609E9

'2.7.7.8': 'MCA1309' = Polyribonucleotide nucleotidyltransferase

Involved in mRNA degradation. Catalyzes the phosphorolysis of single-stranded polyribonucleotides processively in the 3'- to 5'-direction

RNA(n+1) + phosphate = RNA(n) + a nucleoside diphosphate
http://www.uniprot.org/uniprot/Q609C6

'2.8.1.10': 'MCA1077' = Thiazole synthase

Catalyzes the rearrangement of 1-deoxy-D-xylulose 5-phosphate (DXP) to produce the thiazole phosphate moiety of thiamine. Sulfur is provided by the thiocarboxylate moiety of the carrier protein ThiS. In vitro, sulfur can be provided by H2S

1-deoxy-D-xylulose 5-phosphate + 2-iminoacetate + thiocarboxy-[sulfur-carrier protein ThiS] = 2-((2R,5Z)-2-carboxy-4-methylthiazol-5(2H)-ylidene)ethyl phosphate + [sulfur-carrier protein ThiS] + 2 H2O.

This protein is involved in the pathway thiamine diphosphate biosynthesis, which is part of Cofactor biosynthesis

http://www.uniprot.org/uniprot/Q609Z5

2.8.1.2': 'MCA1555' = Mercaptopyruvate sulfurtransferase, sseA

3-mercaptopyruvate + cyanide <=> pyruvate + thiocyanate

http://www.uniprot.org/uniprot/Q608D7

'2.8.1.6': 'MCA1125' = Biotin synthase 

Catalyzes the conversion of dethiobiotin (DTB) to biotin by the insertion of a sulfur atom into dethiobiotin via a radical-based mechanism

Dethiobiotin + sulfur-(sulfur carrier) + 2 S-adenosyl-L-methionine + 2 reduced [2Fe-2S] ferredoxin = biotin + (sulfur carrier) + 2 L-methionine + 2 5'-deoxyadenosine + 2 oxidized [2Fe-2S] ferredoxin.

This protein is involved in step 2 of the subpathway that synthesizes biotin from 7,8-diaminononanoate
http://www.uniprot.org/uniprot/Q609V2

'2.8.1.8': 'MCA0110' = Lipoyl synthase, lipA 
Catalyzes the radical-mediated insertion of two sulfur atoms into the C-6 and C-8 positions of the octanoyl moiety bound to the lipoyl domains of lipoate-dependent enzymes, thereby converting the octanoylated domains into lipoylated derivatives.

Protein N(6)-(octanoyl)lysine + 2 sulfur-(sulfur carrier) + 2 S-adenosyl-L-methionine + 2 reduced [2Fe-2S] ferredoxin = protein N(6)-(lipoyl)lysine + 2 (sulfur carrier) + 2 L-methionine + 2 5'-deoxyadenosine + 2 oxidized [2Fe-2S] ferredoxin

http://www.uniprot.org/uniprot/Q60CJ6

'3.2.1.-': 'MCA0296' = Glycogen debranching enzyme GlgX, glgX

http://www.uniprot.org/uniprot/Q60C15

'3.4.11.5': 'MCA1737' = Proline iminopeptidase
 
Release of N-terminal proline from a peptide
http://www.uniprot.org/uniprot/Q607M2

'3.5.1.11': 'MCA3025' = Penicillin acylase II, acyII
    
Penicillin + H(2)O <=> a carboxylate + 6-aminopenicillanate
http://www.uniprot.org/uniprot/Q602N8


'3.5.2.6': 'MCA2220' = Beta-lactamase

A beta-lactam + H2O = a substituted beta-amino acid

http://www.uniprot.org/uniprot/Q605Q8

'4.2.1.109': 'MCA0799' =  Methylthioribulose-1-phosphate dehydratase, mtnB
    
Catalyzes the dehydration of methylthioribulose-1-phosphate (MTRu-1-P) into 2,3-diketo-5-methylthiopentyl-1-phosphate (DK-MTP-1-P).

S-methyl-5-thio-D-ribulose 1-phosphate = 5-(methylthio)-2,3-dioxopentyl phosphate + H2O.

This subpathway is part of the pathway L-methionine biosynthesis via salvage pathway, which is itself part of Amino-acid biosynthesis.

http://www.uniprot.org/uniprot/Q60AP7

'4.3.99.3': 'MCA1232' = 7-carboxy-7-deazaguanine synthase

Catalyzes the complex heterocyclic radical-mediated conversion of 6-carboxy-5,6,7,8-tetrahydropterin (CPH4) to 7-carboxy-7-deazaguanine (CDG), a step common to the biosynthetic pathways of all 7-deazapurine-containing compounds.

6-carboxy-5,6,7,8-tetrahydropterin = 7-carboxy-7-carbaguanine + NH3

This protein is involved in the pathway 7-cyano-7-deazaguanine biosynthesis, which is part of Purine metabolism

http://www.uniprot.org/uniprot/Q609K1

'6.3.4.20': 'MCA1233' = 7-cyano-7-deazaguanine synthase
    
Catalyzes the ATP-dependent conversion of 7-carboxy-7-deazaguanine (CDG) to 7-cyano-7-deazaguanine (preQ0)

7-carboxy-7-carbaguanine + NH3 + ATP = 7-cyano-7-carbaguanine + ADP + phosphate + H2O.

http://www.uniprot.org/uniprot/Q609K0

They are somewhat metabolism related, but I lack precise information on biotin or pyridoxal phosphate so they are not included in the biomass reaction. At a later stage this should be looked into!

## Interconversion of Pyruvate and PEP and Oxaloacetate revisited.

In [142]:
model.reactions.BIOMASS_REACTION.change_bounds(0.1,1000)

In [143]:
model.change_objective(model.reactions.BIOMASS_REACTION)

In [144]:
model.solve().f

5.981080383363871

Removed the following reactions because the genes they had as GPR did not fulfill that function.

In [145]:
model.remove_reactions([model.reactions.PPS_c,model.reactions.GTPOPm_c,model.reactions.AGPOP_c, model.reactions.DAPOP_c])

In [146]:
model.solve().f

5.981080383363658

In [147]:
from cameo.util import TimeMachine

In [148]:
rxn = model.reactions.BIOMASS_REACTION

for met in rxn.reactants:
    with TimeMachine() as tm:
        dm = model.add_demand(met, 'TDM', tm)
        model.change_objective(dm)
        try: 
            solution = model.solve()
            if solution.f == 0:
                print '{} cannot be produced'.format(met.id)
        except SolverError:
            print 'With the current bounds the solver cannot solve the problem. It is infeasible.'

# Corrections to the biomass equation based on memote-consistency check

In [149]:
model.reactions.BIOMASS_REACTION.metabolites[model.metabolites.gln__L_c]

-0.15

In [150]:
ct.invert_reaction(model.reactions.DTMPK_c)

In [151]:
model.reactions.BIOMASS_REACTION.add_metabolites({model.metabolites.ala__L_c: -0.576,
                                                 model.metabolites.arg__L_c: -0.254,
                                                 model.metabolites.asp__L_c: -0.468,
                                                 model.metabolites.cys__L_c: -0.038,
                                                 model.metabolites.glu__L_c: -0.519,
                                                
                                                 model.metabolites.gly_c: -0.484,
                                                 model.metabolites.his__L_c: -0.104,
                                                 model.metabolites.ile__L_c: -0.242,
                                                 model.metabolites.leu__L_c: -0.415,
                                                 model.metabolites.lys__L_c: -0.277,
                                                 model.metabolites.met__L_c: -0.130,
                                                 model.metabolites.phe__L_c: -0.185,
                                                 model.metabolites.pro__L_c: -0.248,
                                                 model.metabolites.ser__L_c: -0.246,
                                                 model.metabolites.thr__L_c: -0.271,
                                                 model.metabolites.trp__L_c: -0.093,
                                                 model.metabolites.tyr__L_c: -0.132,
                                                 model.metabolites.val__L_c: -0.360,
                                                 model.metabolites.amp_c: 0,
                                                 model.metabolites.atp_c: -(23.087 + 0.024),
                                                 model.metabolites.ump_c: 0,
                                                 model.metabolites.utp_c: -0.026,
                                                 model.metabolites.gmp_c: 0,
                                                 model.metabolites.gtp_c: -0.041,
                                                 model.metabolites.cmp_c: 0,
                                                 model.metabolites.ctp_c: -0.044,
                                                 model.metabolites.damp_c: 0,
                                                 model.metabolites.datp_c: -0.009,
                                                 model.metabolites.dtmp_c: 0,
                                                 model.metabolites.dttp_c: -0.009,
                                                 model.metabolites.dgmp_c: 0,
                                                 model.metabolites.dgtp_c: -0.014,
                                                 model.metabolites.dcmp_c: 0,
                                                 model.metabolites.dctp_c: -0.016,
                                                 model.metabolites.h2o_c: -17.776,
                                                 model.metabolites.adp_c: 23.087,
                                                 model.metabolites.pi_c: 22.887,
                                                 model.metabolites.h_c: 23.087,
                                                 model.metabolites.ppi_c: 0.183}
                                                 ,combine=False)

In [152]:
rxn = model.reactions.BIOMASS_REACTION

for met in rxn.reactants:
    with TimeMachine() as tm:
        dm = model.add_demand(met, 'TDM', tm)
        model.change_objective(dm)
        try: 
            model.solve().f
        except:
            print '{} cannot be produced'.format(met.id)

In [153]:
model.change_objective(model.reactions.BIOMASS_REACTION)

In [154]:
model.solve().f

6.194064622533217

In [155]:
rxn = model.reactions.BIOMASS_REACTION
control_sum = 0

for met in rxn.metabolites:
    control_sum += (rxn.metabolites[met]/1000) * met.formula_weight
    print abs(control_sum)
   

0.012482842398
0.013223223581
0.013964612704
0.058823426824
0.077816368564
0.078044462485
0.097441936685
0.097513445826
0.121430333106
0.129772608034
0.129907791742
0.129943822918
0.132077409918
0.160629763998
0.176351796478
0.177449997004
0.177988882444
2.01868937954
2.01860150157
2.01827201607
9.61001060112
9.61011099676
0.182870374517
0.182702403998
0.182621850093
0.178237501326
0.145956208966
0.145928063624
0.0700910985442
0.0626810959362
0.0219079424362
0.0165686084362
0.0163795087362
0.0199527256638
0.0412378208468
0.0425626551708
0.0938803268508
0.0939802789934
0.0707099682134
0.0720157800134
0.0728408707184
0.0733571105204
0.3935967278
0.394923578004
0.395045679464
0.395584564904
0.416666045676
0.420752703916
0.462925386316
0.462992775735
0.494736622375
0.494908999825
0.556729338145
0.556777616057
0.609298563897
0.609343633317
0.609457689597
0.609618097172
0.610122223055
0.610650509295
0.578634428349
0.578970512271
0.609530503171
0.614134514011
0.658637574611
0.658638339899
0.6

# Improvement of Lipid and Fatty Acid incorporation into Biomass.

In [156]:
model_x = load_model('/Users/clie/Dev/cameo/tests/data/iMM904.xml')

In [157]:
model_x.reactions.TRIGS_SC.check_mass_balance()

{'C': 7.105427357601002e-15,
 'H': -7.105427357601002e-15,
 'N': -2.220446049250313e-16,
 'S': 2.7755575615628914e-17}

In [158]:
model.reactions.MC_Average_FattyAcid_c

Id,MC_Average_FattyAcid_c
Name,Reaction added to get an acurate representation of the average fatty acid composition of MC
Stoichiometry,0.092 cpoa2h_c + 0.729 hdca_c + 0.089 hpdca_c + 0.004 ocdca_c + 0.007 ocdcea_c + 0.01 ptdca_c + 0.061 ttdca_c + 0.007 ttdcea_c <=> mc_fattyacid_c
GPR,
Lower bound,-1000.000000
Upper bound,1000.000000


In [159]:
storage = []
for met,coef in model.reactions.MC_Average_FattyAcid_c.metabolites.items():
    if coef < 0:
        storage.append({e:v*coef for e,v in met.elements.iteritems()})

In [160]:
storage

[{'C': -0.15, 'H': -0.29, 'O': -0.02},
 {'C': -0.098, 'H': -0.17500000000000002, 'O': -0.014},
 {'C': -1.513, 'H': -2.937, 'O': -0.178},
 {'C': -1.564, 'H': -3.036, 'O': -0.184},
 {'C': -11.664, 'H': -22.599, 'O': -1.458},
 {'C': -0.126, 'H': -0.231, 'O': -0.014},
 {'C': -0.07200000000000001, 'H': -0.14, 'O': -0.008},
 {'C': -0.854, 'H': -1.647, 'O': -0.122}]

In [161]:
average = {'H':0,'C':0,'O':0}
for di in storage:
    for e,v in di.items():
        average[e] += v

In [162]:
average = {k : v*-1 for (k,v) in average.items()} 

In [163]:
def formula_changer(rxn, met1_id, met2_id, factor,float_ok):
    rxn_obj = model.reactions.get_by_id(rxn)
    met1_obj = model.metabolites.get_by_id(met1_id)
    met2_obj = model.metabolites.get_by_id(met2_id)
    
    met2_obj.formula = ''
    met2_obj.charge = 0
    
    coef1 = rxn_obj.metabolites[met1_obj]
    coef2 = rxn_obj.metabolites[met2_obj]
    
    rxn_obj.add_metabolites({met1_obj: coef1/factor,
                             met2_obj: coef2/factor},combine=False)
    
    imba_dict =  rxn_obj.check_mass_balance()
    print imba_dict
    if 'charge' in imba_dict.keys():
        if float_ok == 'ok':
            met2_obj.charge = -imba_dict['charge']*factor
        else:
            met2_obj.charge = -int(imba_dict['charge']*factor)
        imba_dict.pop('charge', 0)
    formula = ''
    for k,v in sorted(imba_dict.items()):
        if float_ok == 'ok':
            formula = formula + '{}{}'.format(k,abs(v*factor))
        else:
            formula = formula + '{}{}'.format(k,abs(int(v*factor)))
    met2_obj.formula = formula
    print formula
    print rxn_obj.check_mass_balance()

# Fatty acid as float (will lead to an invalid SMBL file)

In [164]:
# model.metabolites.mc_fattyacid_c.elements = average

In [165]:
# model.metabolites.mc_fattyacid_c.formula

In [166]:
# model.metabolites.mc_fattyacid_c.formula_weight

In [167]:
# model.reactions.MC_Average_FattyAcid_c.check_mass_balance()

In [168]:
# formula_changer('MC_AFAA_c','mc_fattyacid_c','mc_fattyacidcoa_c',1)

In [169]:
# formula_changer('G3PAT_MC_c','mc_fattyacidcoa_c','1agpgafa_c',1)

In [170]:
# formula_changer('APG3PAT_MC_c','mc_fattyacidcoa_c','pa_MC_c',1)

In [171]:
# formula_changer('PHCYT_MC_c','pa_MC_c','cdpdag_MC_c',1)

In [172]:
# formula_changer('PGSA_MC_c','cdpdag_MC_c','pgp_MC_c',1)

In [173]:
# formula_changer('PGPP_MC_c','pgp_MC_c','pg_MC_c',1)

In [174]:
# formula_changer('CLPNS_MC_c','pg_MC_c','clpn_MC_c',1)

In [175]:
# formula_changer('PSSA_MC_c','cdpdag_MC_c','ps_MC_c',1)

In [176]:
# formula_changer('PSD_MC_c','ps_MC_c','pe_MC_c',1)

In [177]:
# formula_changer('PETOHM_MC_c','pe_MC_c','pme_c',1)

In [178]:
# formula_changer('PMETM_MC_c','pme_c','pdme_c',1)

In [179]:
# formula_changer('PMETM2_MC_c','pdme_c','pc_MC_c',1)

In [180]:
# model.metabolites.clpn_MC_c.formula_weight

In [181]:
# model.metabolites.pg_MC_c.formula_weight

In [182]:
# model.metabolites.pe_MC_c.formula_weight

In [183]:
# model.metabolites.pc_MC_c.formula_weight

In [184]:
# model.reactions.BIOMASS_REACTION.add_metabolites({model.metabolites.pc_MC_c: -0.009,
#                                                  model.metabolites.pe_MC_c: -0.091,
#                                                  model.metabolites.clpn_MC_c: -0.003,
#                                                  model.metabolites.pg_MC_c: -0.015}
#                                                  ,combine=False)

In [185]:
# rxn = model.reactions.BIOMASS_REACTION
# control_sum = 0

# for met in rxn.metabolites:
#     control_sum += (rxn.metabolites[met]/1000) * met.formula_weight
#     print abs(control_sum)

# Lipid fixes with the iMM904 way

In [186]:
average

{'C': 16.041, 'H': 31.055, 'O': 1.9980000000000002}

In [187]:
model.metabolites.mc_fattyacid_c.elements = {'C': 16041, 'H': 31055, 'O': 1998}

In [188]:
formula_changer('MC_AFAA_c','mc_fattyacid_c','mc_fattyacidcoa_c',1000,'no')

{'C': -37.041, 'H': -62.05500000000001, 'charge': 3.0010000000000003, 'O': -16.998, 'N': -7.0, 'P': -3.0, 'S': -1.0}
C37041H62055N7000O16998P3000S1000
{'C': 3.552713678800501e-15, 'charge': 4.440892098500626e-16, 'O': 8.881784197001252e-16}


In [189]:
formula_changer('G3PAT_MC_c','mc_fattyacidcoa_c','1agpgafa_c',1000,'no')

{'H': -37.055, 'C': -19.041000000000004, 'charge': 1.001, 'O': -6.998000000000001, 'P': -1.0}
C19041H37055O6998P1000
{'C': -7.105427357601002e-15, 'charge': 0.0009999999999998899}


In [190]:
model.reactions.APG3PAT_MC_c

Id,APG3PAT_MC_c
Name,Acyl-phosphate:glycerol-3-phosphate acyltransferase
Stoichiometry,1agpgafa_c + mc_fattyacidcoa_c --> coa_c + pa_MC_c
GPR,MCA0058 or MCA2052
Lower bound,0.000000
Upper bound,1000.000000


In [191]:
model.reactions.APG3PAT_MC_c.add_metabolites({model.metabolites.get_by_id('1agpgafa_c'):-0.001},combine=False)

In [192]:
formula_changer('APG3PAT_MC_c','mc_fattyacidcoa_c','pa_MC_c',1000,'no')

{'H': -67.11, 'C': -35.08200000000001, 'charge': 0.0009999999999998899, 'O': -7.996000000000001, 'P': -1.0}
C35082H67110O7996P1000
{'C': -3.552713678800501e-15, 'charge': 0.0009999999999998899, 'O': -8.881784197001252e-16}


In [193]:
formula_changer('PHCYT_MC_c','pa_MC_c','cdpdag_MC_c',1000,'no')

{'H': -79.11, 'C': -44.082, 'P': -2.0, 'O': -14.996000000000002, 'N': -3.0}
C44082H79110N3000O14996P2000
{'O': -1.7763568394002505e-15}


In [194]:
formula_changer('PGSA_MC_c','cdpdag_MC_c','pgp_MC_c',1000,'no')

{'H': -73.11, 'C': -38.082, 'charge': 1.0, 'O': -12.996000000000002, 'P': -2.0}
C38082H73110O12996P2000
{'O': -1.7763568394002505e-15}


In [195]:
formula_changer('PGPP_MC_c','pgp_MC_c','pg_MC_c',1000,'no')

{'H': -74.11, 'charge': -1.0, 'C': -38.082, 'O': -9.996, 'P': -1.0}
C38082H74110O9996P1000
{}


In [196]:
formula_changer('CLPNS_MC_c','pg_MC_c','clpn_MC_c',1000,'no')

{'H': -140.22, 'charge': -2.0, 'C': -73.164, 'O': -16.992, 'P': -2.0}
C73164H140220O16992P2000
{}


In [197]:
formula_changer('PSSA_MC_c','cdpdag_MC_c','ps_MC_c',1000,'no')

{'C': -38.082, 'H': -73.11, 'O': -9.996, 'N': -1.0, 'P': -1.0, 'charge': -1.0}
C38082H73110N1000O9996P1000
{}


In [198]:
formula_changer('PSD_MC_c','ps_MC_c','pe_MC_c',1000,'no')

{'C': -37.082, 'H': -74.11, 'O': -7.996, 'N': -1.0, 'P': -1.0, 'charge': -2.0}
C37082H74110N1000O7996P1000
{}


In [199]:
formula_changer('PETOHM_MC_c','pe_MC_c','pme_c',1000,'no')

{'C': -38.082, 'H': -76.11, 'O': -7.996, 'N': -1.0, 'P': -1.0, 'charge': -2.0}
C38082H76110N1000O7996P1000
{}


In [200]:
formula_changer('PMETM_MC_c','pme_c','pdme_c',1000,'no')

{'C': -39.082, 'H': -78.11, 'O': -7.996, 'N': -1.0, 'P': -1.0, 'charge': -2.0}
C39082H78110N1000O7996P1000
{}


In [201]:
formula_changer('PMETM2_MC_c','pdme_c','pc_MC_c',1000,'no')

{'C': -40.082, 'H': -75.11, 'O': -7.996, 'N': -1.0, 'P': -1.0, 'charge': -3.0}
C40082H75110N1000O7996P1000
{}


Add copper as a metabolite and add it to the biomass reaction.

In [242]:
met = Metabolite(id='cu_c',formula='Cu',name='Copper',charge=2,compartment='c')

In [243]:
rxn = Reaction(id='EX_cu_c',name='Cu+ exchange', lower_bound=-1000, upper_bound=1000)

In [244]:
rxn.add_metabolites({met:-1})

In [245]:
model.add_reactions([rxn])

In [246]:
model.reactions.BIOMASS_REACTION.add_metabolites({model.metabolites.pc_MC_c: -(0.00937385/1000),
                                                  model.metabolites.pe_MC_c: -(0.09134262/1000),
                                                  model.metabolites.clpn_MC_c: -(0.00315904/1000),
                                                  model.metabolites.pg_MC_c: -(0.01538077/1000),
                                                  model.metabolites.utp_c: -0.02560771,
                                                  model.metabolites.gtp_c: -0.04084668,
                                                  model.metabolites.ctp_c: -0.04425885,
                                                  model.metabolites.datp_c: -0.00866366,
                                                  model.metabolites.dttp_c: -0.00882698,
                                                  model.metabolites.dgtp_c: -0.01446787,
                                                  model.metabolites.dctp_c: -0.01571821,
                                                  model.metabolites.sql_c: -0.01339118,
                                                  model.metabolites.dpterol_c: -0.01282849,
                                                  model.metabolites.lanost_c: -0.00515564,
                                                  model.metabolites.ttdca_c: -0.00145650,
                                                  model.metabolites.ttdcea_c: -0.00017921,
                                                  model.metabolites.ptdca_c: -0.00023422,
                                                  model.metabolites.hdca_c: -0.01558994,
                                                  model.metabolites.hpdca_c: -0.00179858,
                                                  model.metabolites.cpoa2h_c: -0.00186552,
                                                  model.metabolites.ocdca_c: -0.00008548,
                                                  model.metabolites.ocdcea_c: -0.00014349,
                                                  model.metabolites.glc__D_c: -0.24977798,
                                                  model.metabolites.glc__D_c: -0.24977798,
                                                  model.metabolites.thmpp_c: -0.00002865,
                                                  model.metabolites.ribflv_c: -0.00019396,
                                                  model.metabolites.nac_c: -0.00106469,
                                                  model.metabolites.inost_c: -0.00016652,
                                                  model.metabolites.atp_c: -23.11185589,
                                                  model.metabolites.adp_c: 23.08742079,
                                                  model.metabolites.pi_c: 22.88736056,
                                                  model.metabolites.ppi_c: 0.18282505,
                                                  model.metabolites.h_c: 23.08742079,
                                                  model.metabolites.h2o_c: -17.77579456,
                                                  model.metabolites.cl_c: -0.21438646,
                                                  model.metabolites.ca2_c: -0.06986377,
                                                  model.metabolites.k_c: -0.17647826,
                                                  model.metabolites.mg2_c: -0.12343139,
                                                  model.metabolites.na1_c: -0.03914785,
                                                  model.metabolites.cu_c: -0.00151072,
                                                  model.metabolites.cobalt2_c: -0.00005769}
                                                 ,combine=False)

In [247]:
rxn = model.reactions.BIOMASS_REACTION
control_sum = 0

for met in rxn.metabolites:
    control_sum += (rxn.metabolites[met]/1000) * met.formula_weight

print abs(control_sum)

0.902342720019


In [249]:
model.reactions.EX_cl_c.lower_bound = -1000
model.reactions.EX_ca2_c.lower_bound = -1000
model.reactions.EX_k_c.lower_bound = -1000
model.reactions.EX_mg2_c.lower_bound = -1000
model.reactions.EX_na1_c.lower_bound = -1

In [256]:
model.reactions.EX_ch4_p.lower_bound = -18.46

In [257]:
model.solve().f

0.26026613903477824

# Saving and Export

In [251]:
import cobra

In [252]:
target_filename_json = relative_directory + '/Reconstructions/MethylococcusModel11.json'
target_filename_xml = relative_directory + '/Reconstructions/MethylococcusModel11.xml'
cobra.io.write_sbml_model(model, target_filename_xml, use_fbc_package=True)
cobra.io.save_json_model(model, target_filename_json,pretty = True)

# Validation Implementation 2.0

In [4]:
relative_directory = getcwd()

In [5]:
target_filename_xml = relative_directory + '/Reconstructions/MethylococcusModel11.xml'

In [6]:
xyz = load_model(target_filename_xml)

In [7]:
relative_directory = getcwd()
model = load_model(relative_directory + '/Reconstructions/MethylococcusModel11.json')

In [8]:
model.reactions.pMMO_im.bounds

(0, 1000)

In [9]:
model.reactions.pMMO_dir_coupl_im.bounds

(0, 0)

In [10]:
model.reactions.CYOR_q8_im.bounds

(0, 1000)

In [11]:
model.reactions.sMMO_c.bounds

(0, 0)

In [12]:
import cameo

In [13]:
def efficiency_prediction(n_source,lim,ratio):
    o2_utr = []
    ch4_utr = []
    grwth_rate = []
    
    for l in range(len(n_source)):
        
        if not ratio[l] == 100:
            model.reactions.sMMO_c.change_bounds(0,1000)
            model.reactions.pMMO_im.change_bounds(0,1000)
            constraint1 = model.add_ratio_constraint('pMMO_im','sMMO_c',ratio[l])
        
        else:
            model.reactions.pMMO_im.change_bounds(0,1000)
            model.reactions.sMMO_c.change_bounds(0,0)
            model.reactions.CYTCBD_im.change_bounds(0,0)
            model.reactions.PPApt_im.change_bounds(0,0)
            
#####################################################################################
        
        if n_source[l] == 'no3':
            model.reactions.EX_nh3_c.lower_bound = 0
            model.reactions.EX_no3_e.lower_bound = -1000
            model.reactions.GLNS_c.upper_bound = 1000
            model.reactions.GLNS_c.lower_bound = 0
        
        elif n_source[l] == 'nh4':
            model.reactions.EX_nh3_c.lower_bound = -1000
            model.reactions.EX_no3_e.lower_bound = 0
            model.reactions.GLNS_c.upper_bound = 0
            model.reactions.GLNS_c.lower_bound = 0
        
        elif n_source[l] == 'both':
            model.reactions.EX_nh3_c.lower_bound = -1000
            model.reactions.EX_no3_e.lower_bound = -1000
            model.reactions.GLNS_c.upper_bound = 1000
            model.reactions.GLNS_c.lower_bound = 0
            constraint2 = model.add_ratio_constraint('EX_nh3_c','EX_no3_e',4.12)
        
#####################################################################################

        if lim[l] == 'O2':
            model.reactions.EX_o2_c.lower_bound = -100
            model.reactions.EX_ch4_p.lower_bound = -1000
            model.reactions.EX_co2_c.lower_bound = -1000           
            
        elif lim[l] == 'CH4':
            model.reactions.EX_o2_c.lower_bound = -1000
            model.reactions.EX_ch4_p.lower_bound = -100
            model.reactions.EX_co2_c.lower_bound = -1000


#####################################################################################
            
        model.reactions.BIOMASS_REACTION.lower_bound = 0.1
        model.reactions.BIOMASS_REACTION.upper_bound = 0.1
            
        try:
            model.change_objective(model.reactions.BIOMASS_REACTION)
            solution = cameo.pfba(model)
            o2_utr.append(solution.fluxes['EX_o2_c'])
            ch4_utr.append(solution.fluxes['EX_ch4_p'])
            grwth_rate.append(solution.fluxes['BIOMASS_REACTION'])
        except cameo.exceptions.SolveError:
            o2_utr.append(0)
            ch4_utr.append(0)
            grwth_rate.append(0)
                
        if not ratio[l] == 100:
            model.solver.remove([constraint1])
            
        if n_source[l] == 'both':   
            model.solver.remove([constraint2])
        
    return o2_utr, ch4_utr, grwth_rate

In [14]:
from optlang import Model, Variable, Constraint, Objective

In [15]:
def efficiency_prediction_alt_2(n_source,lim,ratio):
    o2_utr = []
    ch4_utr = []
    grwth_rate = []
    
    for l in range(len(n_source)):
        
        if not ratio[l] == 100:
            model.reactions.sMMO_c.change_bounds(0,1000)
            c1 = Constraint(((1.0*model.solver.variables.pMMO_dir_coupl_im - 
                1.0*model.solver.variables.pMMO_dir_coupl_im_reverse_025d7) + 
                 (1.0*model.solver.variables.pMMO_im - 
                  1.0*model.solver.variables.pMMO_im_reverse_bce3b)) -
                ratio[l]*(1.0*model.solver.variables.sMMO_c - 
                 1.0*model.solver.variables.sMMO_c_reverse_3e312),ub=0,lb=0)
            model.solver.add([c1])
        
        else:
            model.reactions.sMMO_c.change_bounds(0,0)
            model.reactions.CYTCBD_im.change_bounds(0,0)
            model.reactions.PPApt_im.change_bounds(0,0)
            
#####################################################################################
        
        if n_source[l] == 'no3':
            model.reactions.EX_nh3_c.lower_bound = 0
            model.reactions.EX_no3_e.lower_bound = -1000
            model.reactions.GLNS_c.upper_bound = 1000
            model.reactions.GLNS_c.lower_bound = 0
        
        elif n_source[l] == 'nh4':
            model.reactions.EX_nh3_c.lower_bound = -1000
            model.reactions.EX_no3_e.lower_bound = 0
            model.reactions.GLNS_c.upper_bound = 0
            model.reactions.GLNS_c.lower_bound = 0
        
        elif n_source[l] == 'both':
            model.reactions.EX_nh3_c.lower_bound = -1000
            model.reactions.EX_no3_e.lower_bound = -1000
            model.reactions.GLNS_c.upper_bound = 1000
            model.reactions.GLNS_c.lower_bound = 0
            constraint2 = model.add_ratio_constraint('EX_nh3_c','EX_no3_e',4.12)
        
#####################################################################################

        if lim[l] == 'O2':
            model.reactions.EX_o2_c.lower_bound = -1000
            model.reactions.EX_ch4_p.lower_bound = -1000
            model.reactions.EX_co2_c.lower_bound = -1000
            
            model.change_objective(model.reactions.EX_o2_c)
            model.objective.direction = 'max'
            
            
        elif lim[l] == 'CH4':
            model.reactions.EX_o2_c.lower_bound = -1000
            model.reactions.EX_ch4_p.lower_bound = -1000
            model.reactions.EX_co2_c.lower_bound = -1000
            
            model.change_objective(model.reactions.EX_ch4_p)
            model.objective.direction = 'max'

#####################################################################################
            
        model.reactions.BIOMASS_REACTION.lower_bound = 0.1
        model.reactions.BIOMASS_REACTION.upper_bound = 0.1
            
        try:
            solution = cameo.pfba(model)
            o2_utr.append(solution.fluxes['EX_o2_c'])
            ch4_utr.append(solution.fluxes['EX_ch4_p'])
            grwth_rate.append(solution.fluxes['BIOMASS_REACTION'])
            print 'Solved'
        except cameo.exceptions.SolveError:
            o2_utr.append(0)
            ch4_utr.append(0)
            grwth_rate.append(0)
            print 'Solver Error!!'
                
        if not ratio[l] == 100:
            model.solver.remove([c1])
            
        if n_source[l] == 'both':   
            model.solver.remove([constraint2])
            
    print o2_utr
    print ch4_utr
    print grwth_rate
    return o2_utr, ch4_utr, grwth_rate

In [16]:
def efficiency_prediction_alt_3(n_source,lim,ratio):
    o2_utr = []
    ch4_utr = []
    grwth_rate = []
    sol_array = []
    
    model.reactions.BIOMASS_REACTION.lower_bound = 0.1
    model.reactions.BIOMASS_REACTION.upper_bound = 0.1
    
    for l in range(len(n_source)):
        
        if not ratio[l] == 100:
            model.reactions.sMMO_c.change_bounds(0,1000)
            c1 = Constraint(((1.0*model.solver.variables.pMMO_dir_coupl_im - 
                1.0*model.solver.variables.pMMO_dir_coupl_im_reverse_025d7) + 
                 (1.0*model.solver.variables.pMMO_im - 
                  1.0*model.solver.variables.pMMO_im_reverse_bce3b)) -
                ratio[l]*(1.0*model.solver.variables.sMMO_c - 
                 1.0*model.solver.variables.sMMO_c_reverse_3e312),ub=0,lb=0)
            model.solver.add([c1])
        
        else:
            model.reactions.sMMO_c.change_bounds(0,0)
            model.reactions.CYTCBD_im.change_bounds(0,0)
            model.reactions.PPApt_im.change_bounds(0,0)
            
#####################################################################################
        
        if n_source[l] == 'no3':
            model.reactions.EX_nh3_c.lower_bound = 0
            model.reactions.EX_no3_e.lower_bound = -1000
            model.reactions.GLNS_c.upper_bound = 1000
            model.reactions.GLNS_c.lower_bound = 0
        
        elif n_source[l] == 'nh4':
            model.reactions.EX_nh3_c.lower_bound = -1000
            model.reactions.EX_no3_e.lower_bound = 0
            model.reactions.GLNS_c.upper_bound = 0
            model.reactions.GLNS_c.lower_bound = 0
        
        elif n_source[l] == 'both':
            model.reactions.EX_nh3_c.lower_bound = -1000
            model.reactions.EX_no3_e.lower_bound = -1000
            model.reactions.GLNS_c.upper_bound = 1000
            model.reactions.GLNS_c.lower_bound = 0
            constraint2 = model.add_ratio_constraint('EX_nh3_c','EX_no3_e',4.12)
        
#####################################################################################

        if lim[l] == 'O2':
            model.reactions.EX_o2_c.change_bounds(-1000,0)
            model.reactions.EX_ch4_p.change_bounds(-1000,0)
            model.reactions.EX_co2_c.lower_bound = -1000
            
            model.change_objective(model.reactions.EX_o2_c)
            model.objective.direction = 'max'
            
            try:
                solution = model.solve()
                o2_utr.append(model.reactions.EX_o2_c.flux)
                model.reactions.EX_o2_c.change_bounds(model.reactions.EX_o2_c.flux,model.reactions.EX_o2_c.flux)
                
                model.change_objective(model.reactions.EX_ch4_p)
                model.objective.direction = 'max'
                solution = model.solve()
                
                ch4_utr.append(model.reactions.EX_ch4_p.flux)
                grwth_rate.append(model.reactions.BIOMASS_REACTION.flux)
                sol_array.append(model.solution.fluxes)

                print 'Solved'
            except cameo.exceptions.SolveError:
                o2_utr.append(0)
                ch4_utr.append(0)
                sol_array.append(0)
                grwth_rate.append(0)
                print 'Solver Error!!'
            
            
        elif lim[l] == 'CH4':
            model.reactions.EX_o2_c.change_bounds(-1000,0)
            model.reactions.EX_ch4_p.change_bounds(-1000,0)
            model.reactions.EX_co2_c.lower_bound = -1000
            
            model.change_objective(model.reactions.EX_ch4_p)
            model.objective.direction = 'max'
            
            try:
                solution = model.solve()
                ch4_utr.append(model.reactions.EX_ch4_p.flux)
                
                model.reactions.EX_ch4_p.change_bounds(model.reactions.EX_ch4_p.flux,model.reactions.EX_ch4_p.flux)
                
                model.change_objective(model.reactions.EX_o2_c)
                model.objective.direction = 'max'
                solution = model.solve()
                
                o2_utr.append(model.reactions.EX_o2_c.flux)
                grwth_rate.append(model.reactions.BIOMASS_REACTION.flux)
                sol_array.append(solution.fluxes)
                print 'Solved'
            except cameo.exceptions.SolveError:
                o2_utr.append(0)
                ch4_utr.append(0)
                sol_array.append(0)
                grwth_rate.append(0)
                print 'Solver Error!!'

#####################################################################################
                
        if not ratio[l] == 100:
            model.solver.remove([c1])
            
        if n_source[l] == 'both':   
            model.solver.remove([constraint2])
            

    return o2_utr, ch4_utr, grwth_rate, sol_array

Leak and Dalton - Conversion of Copper used.

In [17]:
# Molar Mass of CuSO4.5H2O in g/mol
M_cuso4 = 249.72

In [18]:
((0.2/1000)/ 249.72) *1000000
# µmol/l

0.8008970046452026

In [19]:
((1.5/1000)/ 249.72) *1000000

6.00672753483902

In [20]:
((0.05/1000)/ 249.72) *1000000

0.20022425116130066

In [21]:
((0.1/1000)/ 249.72) *1000000

0.4004485023226013

In [22]:
((1.6/1000)/ 249.72) *1000000

6.407176037161621

In [23]:
((1.2/1000)/ 249.72) *1000000

4.805382027871215

In [24]:
((0.00256)/ 249.72) *1000000

10.251481659458594

In [25]:
# Experimental parameters used by Leak and Dalton, 
# here serve as input parameters for the Efficiency Prediction function.

n_sources = ['no3','no3','no3','no3','nh4','nh4','nh4','both','both']
lim = ['O2','O2','CH4','CH4','O2','O2','CH4','CH4','CH4']
ratio = [0.02,100,0.01,99,0.02,99,24,100,49]
cu_values = [0.8,6,0.2,6,0.4,6,6,6.4,0.2]


# Experimental results as measured by Leak and Dalton
o2_ch4_yields_exp = [1.47,1.45,1.50,1.41,1.56,1.60,1.60,1.48,1.52]
ych4_yields_exp = [0.50,0.69,0.50,0.67,0.71,'0.69-0.75',0.67,0.79,0.60]

### Initial Calculation with previously-found proportions in electron transfer

In [26]:
model.reactions.pMMO_dir_coupl_im.upper_bound = 0

In [27]:
model.reactions.CYOR_q8_im.lower_bound = -1.76

In [28]:
o2_utr, ch4_utr, grwth_rate = efficiency_prediction(n_sources,lim,ratio)
# mmol O2/ mmol CH4
ratios = [abs(float(o2_utr[i]/ch4_utr[i])) for i in range(len(o2_utr))]
# gDW/g Methane
yields = [abs(grwth_rate[i]/((ch4_utr[i]/1000)*16.04)) for i in range(len(o2_utr))]

In [29]:
d = {'Nitrogen Source' : pd.Series(n_sources, index=range(len(n_sources))),
     'CuSO4.5H20 (µM)':pd.Series(cu_values,index=range(len(n_sources))),
     'Limitation' : pd.Series(lim, index=range(len(n_sources))),
    'P/S (%)' : pd.Series(ratio, index=range(len(n_sources))),
     'O2/CH4 - Experimental' : pd.Series(o2_ch4_yields_exp, index=range(len(n_sources))),
     'YCH4 - Experimental' : pd.Series(ych4_yields_exp, index=range(len(n_sources))),
    'O2/CH4 - Predicted' : pd.Series(ratios, index=range(len(n_sources))),
    'YCH4 - Predicted' : pd.Series(yields, index=range(len(n_sources)))}
df = pd.DataFrame(d)
df

,CuSO4.5H20 (µM),Limitation,Nitrogen Source,O2/CH4 - Experimental,O2/CH4 - Predicted,P/S (%),YCH4 - Experimental,YCH4 - Predicted
0,0.8,O2,no3,1.47,1.519655,0.02,0.5,0.584947
1,6.0,O2,no3,1.45,1.468857,100.00,0.69,0.646807
2,0.2,CH4,no3,1.50,1.519655,0.01,0.5,0.584947
3,6.0,CH4,no3,1.41,1.470082,99.00,0.67,0.645314
4,0.4,O2,nh4,1.56,1.599529,0.02,0.71,0.717308
5,6.0,O2,nh4,1.60,1.540573,99.00,0.69-0.75,0.822906
6,6.0,CH4,nh4,1.60,1.543761,24.00,0.67,0.817196
7,6.4,CH4,both,1.48,1.532037,100.00,0.79,0.814912
8,0.2,CH4,both,1.52,1.534197,49.00,0.6,0.811151


In [30]:
model.reactions.CYOR_q8_im.lower_bound = 0

### Initial Calculation with unchanged proportions in electron transfer alt_2

In [31]:
o2_utr, ch4_utr, grwth_rate = efficiency_prediction_alt_2(n_sources,lim,ratio)
ratios = [abs(float(o2_utr[i]/ch4_utr[i])) for i in range(len(o2_utr))]
yields = [abs(grwth_rate[i]/((ch4_utr[i]/1000)*16.04)) for i in range(len(o2_utr))]

Solved
Solved
Solved
Solved
Solved
Solved
Solved
Solved
Solved
[-15.621523941906439, -15.621523941907027, -15.638264920238093, -15.621523941907256, -13.08932888259566, -10.704819087906982, -10.704819087907044, -11.003162829047248, -11.003162829047824]
[-10.370542883764053, -10.370542883764417, -10.378913372929846, -10.370542883764458, -8.284994545108406, -7.09273964776428, -7.09273964776432, -7.291635475191277, -7.291635475191507]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]


In [32]:
d = {'Nitrogen Source' : pd.Series(n_sources, index=range(len(n_sources))),
    'CuSO4.5H20 (µM)':pd.Series(cu_values,index=range(len(n_sources))),
     'Limitation' : pd.Series(lim, index=range(len(n_sources))),
    'P/S (%)' : pd.Series(ratio, index=range(len(n_sources))),
     'O2/CH4 - Experimental' : pd.Series(o2_ch4_yields_exp, index=range(len(n_sources))),
     'YCH4 - Experimental' : pd.Series(ych4_yields_exp, index=range(len(n_sources))),
    'O2/CH4 - Predicted' : pd.Series(ratios, index=range(len(n_sources))),
    'YCH4 - Predicted' : pd.Series(yields, index=range(len(n_sources)))}
df = pd.DataFrame(d)
df

,CuSO4.5H20 (µM),Limitation,Nitrogen Source,O2/CH4 - Experimental,O2/CH4 - Predicted,P/S (%),YCH4 - Experimental,YCH4 - Predicted
0,0.8,O2,no3,1.47,1.506336,0.02,0.5,0.601166
1,6.0,O2,no3,1.45,1.506336,100.00,0.69,0.601166
2,0.2,CH4,no3,1.50,1.506734,0.01,0.5,0.600681
3,6.0,CH4,no3,1.41,1.506336,99.00,0.67,0.601166
4,0.4,O2,nh4,1.56,1.579884,0.02,0.71,0.752495
5,6.0,O2,nh4,1.60,1.509264,99.00,0.69-0.75,0.878985
6,6.0,CH4,nh4,1.60,1.509264,24.00,0.67,0.878985
7,6.4,CH4,both,1.48,1.509012,100.00,0.79,0.855009
8,0.2,CH4,both,1.52,1.509012,49.00,0.6,0.855009


### Calculation with CYOR_reverse allowed in electron transfer alt_2

In [33]:
model.reactions.CYOR_q8_im.lower_bound = -1.74

In [34]:
o2_utr, ch4_utr, grwth_rate = efficiency_prediction_alt_2(n_sources,lim,ratio)
ratios = [abs(float(o2_utr[i]/ch4_utr[i])) for i in range(len(o2_utr))]
yields = [abs(grwth_rate[i]/((ch4_utr[i]/1000)*16.04)) for i in range(len(o2_utr))]

Solved
Solved
Solved
Solved
Solved
Solved
Solved
Solved
Solved
[-15.591975947921235, -12.141523941907067, -15.63826492023871, -12.14152394190965, -13.089328882601649, -10.068780206494733, -11.346402518039035, -10.09309896257514, -10.144566492376565]
[-10.355768886771466, -8.630542883764374, -10.378913372930306, -8.630542883764136, -8.28499454511162, -6.7747202070581345, -6.726600791142552, -6.836603541955171, -6.862337306855886]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]


In [35]:
d = {'Nitrogen Source' : pd.Series(n_sources, index=range(len(n_sources))),
    'CuSO4.5H20 (µM)':pd.Series(cu_values,index=range(len(n_sources))),
     'Limitation' : pd.Series(lim, index=range(len(n_sources))),
    'P/S (%)' : pd.Series(ratio, index=range(len(n_sources))),
     'O2/CH4 - Experimental' : pd.Series(o2_ch4_yields_exp, index=range(len(n_sources))),
     'YCH4 - Experimental' : pd.Series(ych4_yields_exp, index=range(len(n_sources))),
    'O2/CH4 - Predicted' : pd.Series(ratios, index=range(len(n_sources))),
    'YCH4 - Predicted' : pd.Series(yields, index=range(len(n_sources)))}
df = pd.DataFrame(d)
df

,CuSO4.5H20 (µM),Limitation,Nitrogen Source,O2/CH4 - Experimental,O2/CH4 - Predicted,P/S (%),YCH4 - Experimental,YCH4 - Predicted
0,0.8,O2,no3,1.47,1.505632,0.02,0.5,0.602023
1,6.0,O2,no3,1.45,1.406809,100.00,0.69,0.722366
2,0.2,CH4,no3,1.50,1.506734,0.01,0.5,0.600681
3,6.0,CH4,no3,1.41,1.406809,99.00,0.67,0.722366
4,0.4,O2,nh4,1.56,1.579884,0.02,0.71,0.752495
5,6.0,O2,nh4,1.60,1.486228,99.00,0.69-0.75,0.920247
6,6.0,CH4,nh4,1.60,1.686796,24.00,0.67,0.926830
7,6.4,CH4,both,1.48,1.476332,100.00,0.79,0.911917
8,0.2,CH4,both,1.52,1.478296,49.00,0.6,0.908497


In [36]:
model.reactions.CYOR_q8_im.lower_bound = 0

### Test box for fitting

In [37]:
def mean_absolute_error(o2_utr,ch4_utr,grwth_rate,o2_ch4_yields_exp):
    sum_of_errors = 0
    fitness = 99
    try:
        ratios = [abs(float(o2_utr[i]/ch4_utr[i])) for i in range(len(o2_utr))]
        yields = [grwth_rate[i]/((ch4_utr[i]/1000)*16.04) for i in range(len(o2_utr))]
    
        for x in range(len(ratios)):
            sum_of_errors = sum_of_errors + abs((ratios[x] - o2_ch4_yields_exp[x]))    
    
        fitness = sum_of_errors/len(ratios)
        print fitness
        return fitness
    
    except ZeroDivisionError:
        print fitness
        return fitness

In [38]:
def prcnt(number,percent):
    return number+(number*(float(percent)/100))

In [39]:
def hard_evaluation_methylococcus_model(tup):
    #print tup
#     model.reactions.pMMO_im.change_bounds(lb=prcnt(tup[0],-5), ub=prcnt(tup[0],5))
#     model.reactions.pMMO_dir_coupl_im.change_bounds(lb=prcnt(tup[1],-5), ub=prcnt(tup[1],5))
#     model.reactions.CYOR_q8_im.change_bounds(lb=prcnt(tup[2],-5), ub=prcnt(tup[2],5))
    model.reactions.pMMO_im.upper_bound = tup[0]
    model.reactions.pMMO_dir_coupl_im.upper_bound = tup[1]
    model.reactions.CYOR_q8_im.lower_bound = tup[2]
    
    o2_utr, ch4_utr, grwth_rate = efficiency_prediction_alt_2(n_sources,lim,ratio)
    fitness = mean_absolute_error(o2_utr,ch4_utr,grwth_rate,o2_ch4_yields_exp)
    
    return fitness, o2_utr, ch4_utr, grwth_rate

In [40]:
def hard_evaluation_methylococcus_model_2(tup):
    model.reactions.pMMO_im.upper_bound = tup[0]
    model.reactions.pMMO_dir_coupl_im.upper_bound = tup[1]
    model.reactions.CYOR_q8_im.lower_bound = tup[2]
    
    o2_utr, ch4_utr, grwth_rate, sol_array = efficiency_prediction_alt_3(n_sources,lim,ratio)
    fitness = mean_absolute_error(o2_utr,ch4_utr,grwth_rate,o2_ch4_yields_exp)
    
    return fitness, o2_utr, ch4_utr, grwth_rate, sol_array

In [41]:
# # Subselection of those where pMMO is (most) active. This is going to be the training set.
# # I will also focus on fitting the values to NO3 and Both Nitrogen sources, not to pure
# # NH4, to avoid fitting to the 'toxic' effects of NO2.
# n_sources = [n_sources[1],n_sources[3],n_sources[7],n_sources[8]]
# lim = [lim[1],lim[3],lim[7],lim[8]]
# ratio = [ratio[1],ratio[3],ratio[7],ratio[8]]
# cu_values = [cu_values[1],cu_values[3],cu_values[7],cu_values[8]]
# o2_ch4_yields_exp = [o2_ch4_yields_exp[1],o2_ch4_yields_exp[3],o2_ch4_yields_exp[7],o2_ch4_yields_exp[8]]
# ych4_yields_exp = [ych4_yields_exp[1],ych4_yields_exp[3],ych4_yields_exp[7],ych4_yields_exp[8]]

In [42]:
fitness, o2_utr, ch4_utr, grwth_rate = hard_evaluation_methylococcus_model([10.4, 0.0, -1.6])
ratios = [abs(float(o2_utr[i]/ch4_utr[i])) for i in range(len(o2_utr))]
print ratios
yields = [abs(grwth_rate[i]/((ch4_utr[i]/1000)*16.04)) for i in range(len(o2_utr))]
print yields

Solved
Solved
Solved
Solved
Solved
Solved
Solved
Solved
Solved
[-15.591975947921194, -12.421523941907111, -15.638264920238402, -12.421523941907928, -13.089328882596533, -10.068780206494733, -11.231284034629574, -10.09309896257514, -10.144566492376565]
[-10.355768886771408, -8.770542883764367, -10.378913372930075, -8.770542883764804, -8.284994545109024, -6.7747202070581345, -6.734894630004948, -6.836603541955171, -6.862337306855886]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
0.0365576873616
[1.5056318964242803, 1.416277658809613, 1.5067343139239977, 1.4162776588096357, 1.5798838262752637, 1.4862281981777985, 1.6676257984189609, 1.4763323484586117, 1.4782961021518917]
[0.6020232812506277, 0.7108355831231552, 0.6006807977940799, 0.7108355831231198, 0.7524946372798417, 0.9202467075455099, 0.9256884194315453, 0.9119168497672353, 0.9084971615806074]


In [43]:
import cplex

In [44]:
print fitness

0.0365576873616


In [45]:
ratios = [0]*len(o2_utr)
yields = [0]*len(o2_utr)

In [46]:
for i in range(len(o2_utr)):
    try:
        ratios[i] = abs(float(o2_utr[i]/ch4_utr[i]))
    
    except ZeroDivisionError:
        ratios[i] = 0
        
    try:
        yields[i] = grwth_rate[i]/((ch4_utr[i]/1000)*16.04)
        
    except ZeroDivisionError:
        yields[i] = 0

In [47]:
d = {'Nitrogen Source' : pd.Series(n_sources, index=range(len(n_sources))),
    'CuSO4.5H20 (µM)':pd.Series(cu_values,index=range(len(n_sources))),
     'Limitation' : pd.Series(lim, index=range(len(n_sources))),
    'P/S (%)' : pd.Series(ratio, index=range(len(n_sources))),
     'O2/CH4 - Experimental' : pd.Series(o2_ch4_yields_exp, index=range(len(n_sources))),
     'YCH4 - Experimental' : pd.Series(ych4_yields_exp, index=range(len(n_sources))),
    'O2/CH4 - Predicted' : pd.Series(ratios, index=range(len(n_sources))),
    'YCH4 - Predicted' : pd.Series(yields, index=range(len(n_sources)))}
df = pd.DataFrame(d)
df

,CuSO4.5H20 (µM),Limitation,Nitrogen Source,O2/CH4 - Experimental,O2/CH4 - Predicted,P/S (%),YCH4 - Experimental,YCH4 - Predicted
0,0.8,O2,no3,1.47,1.505632,0.02,0.5,-0.602023
1,6.0,O2,no3,1.45,1.416278,100.00,0.69,-0.710836
2,0.2,CH4,no3,1.50,1.506734,0.01,0.5,-0.600681
3,6.0,CH4,no3,1.41,1.416278,99.00,0.67,-0.710836
4,0.4,O2,nh4,1.56,1.579884,0.02,0.71,-0.752495
5,6.0,O2,nh4,1.60,1.486228,99.00,0.69-0.75,-0.920247
6,6.0,CH4,nh4,1.60,1.667626,24.00,0.67,-0.925688
7,6.4,CH4,both,1.48,1.476332,100.00,0.79,-0.911917
8,0.2,CH4,both,1.52,1.478296,49.00,0.6,-0.908497


## ALT 3 No pFBA but instead dual minimzation of just the input flux!

In [346]:
model.reactions.NADH16_im.change_bounds(0,1000)

In [347]:
fitness, o2_utr, ch4_utr, grwth_rate, sol_array = hard_evaluation_methylococcus_model_2([10.4, 0.0, -1.56])
ratios = [abs(float(o2_utr[i]/ch4_utr[i])) for i in range(len(o2_utr))]
print ratios
yields = [abs(grwth_rate[i]/((ch4_utr[i]/1000)*16.04)) for i in range(len(o2_utr))]
print yields

Solved
Solved
Solved
Solved
0.0210574736782
[1.4187621832318174, 1.4187621832318174, 1.476904248108611, 1.4788658571786566]
[0.7095754785698787, 0.709575478569878, 0.9135582619081445, 0.9101324184283501]


In [348]:
ratios = [0]*len(o2_utr)
yields = [0]*len(o2_utr)

In [349]:
for i in range(len(o2_utr)):
    try:
        ratios[i] = abs(float(o2_utr[i]/ch4_utr[i]))
    
    except ZeroDivisionError:
        ratios[i] = 0
        
    try:
        yields[i] = abs(grwth_rate[i]/((ch4_utr[i]/1000)*16.04))
        
    except ZeroDivisionError:
        yields[i] = 0

In [350]:
d = {'Nitrogen Source' : pd.Series(n_sources, index=range(len(n_sources))),
    'CuSO4.5H20 (µM)':pd.Series(cu_values,index=range(len(n_sources))),
     'Limitation' : pd.Series(lim, index=range(len(n_sources))),
    'P/S (%)' : pd.Series(ratio, index=range(len(n_sources))),
     'O2/CH4 - Experimental' : pd.Series(o2_ch4_yields_exp, index=range(len(n_sources))),
     'YCH4 - Experimental' : pd.Series(ych4_yields_exp, index=range(len(n_sources))),
    'O2/CH4 - Predicted' : pd.Series(ratios, index=range(len(n_sources))),
    'YCH4 - Predicted' : pd.Series(yields, index=range(len(n_sources)))}
df = pd.DataFrame(d)
df

,CuSO4.5H20 (µM),Limitation,Nitrogen Source,O2/CH4 - Experimental,O2/CH4 - Predicted,P/S (%),YCH4 - Experimental,YCH4 - Predicted
0,6.0,O2,no3,1.45,1.418762,100,0.69,0.709575
1,6.0,CH4,no3,1.41,1.418762,99,0.67,0.709575
2,6.4,CH4,both,1.48,1.476904,100,0.79,0.913558
3,0.2,CH4,both,1.52,1.478866,49,0.60,0.910132


In [351]:
def calc_percent_distribution(nrofsamples):
    for i in range(nrofsamples):
        rev_elec = sol_array[i]['CYOR_q8_im']
        redox_arm = sol_array[i]['CYTAA3_im']
        total = abs(rev_elec) + abs(redox_arm)
        rev_elec_dist = (abs(rev_elec)/total) * 100
        redox_arm_dist = (abs(redox_arm)/total) * 100

        print('{0:.2f}% vs. {1:.2f}%'.format(redox_arm_dist,rev_elec_dist))

In [352]:
calc_percent_distribution(4)

69.84% vs. 30.16%
69.84% vs. 30.16%
87.69% vs. 12.31%
88.39% vs. 11.61%


In [353]:
(69.84 + 69.84 + 87.69 + 88.39)/4

78.94

In [354]:
(30.16 + 30.16 + 12.31 + 11.61)/4

21.06